In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np

In [9]:
df = pd.read_csv(r'../../results/computed-trajectories-Oct2021toSep2022.csv', delimiter=',', skiprows=0, low_memory=False)
df2 = pd.read_csv(r'../../data/tidy/vehicles-summary.csv', delimiter=',', skiprows=0, low_memory=False)

In [10]:
### Map powertrain
mydict = df2.groupby('Type')['Equipment ID'].agg(list).to_dict()
d = {val:key for key, lst in mydict.items() for val in lst}
df['Powertrain'] = df['Vehicle'].map(d)

In [11]:
### Map vehicle models
mydict = df2.groupby('Model')['Equipment ID'].agg(list).to_dict()
d = {val:key for key, lst in mydict.items() for val in lst}
df['VehicleModel'] = df['Vehicle'].map(d)

In [12]:
# Assign weights based on vehicle models
thisdict = {
  "Gillig 30 ":25700,
  "Gillig 35":26440,
  "Gillig 40":27180,
  "NEW FLYER XD35": 26900 ,
  "NEW FLYER XD40": 28250 ,
  "NEW FLYER XDE40": 28250 ,
  "NEW FLYER XDE60 (ARTIC)": 42250 ,
  "NEW FLYER XE35": 26900 ,
  "NEW FLYER XE40": 32770 ,
  "PROTERRA CATALYST BE-40": 27370 ,
}
df['VehiclWeight(lb)'] = df['VehicleModel'].map(thisdict)

In [ ]:
# map grades
df_trajectories = df.copy()
elevation_dict = df_elevation.set_index('stopid')['elevation'].to_dict()
df_trajectories['elevation'] = df_trajectories['Stop'].map(elevation_dict)
df_trajectories.sort_values(by=['Vehicle', 'ServiceDateTime'], ascending=True, inplace=True)

def calculate_grade(row, prev_row):
    if row.name == 0 or row['dist'] == 0:
        return 0
    elevation_diff = (row['elevation'] - prev_row['elevation']) / 1000
    distance = row['dist'] * 1.61
    if elevation_diff**2 < distance**2:
        return elevation_diff / np.sqrt(distance**2 - elevation_diff**2)
    return 0

df_trajectories['grade'] = df_trajectories.apply(lambda row: calculate_grade(row, df_trajectories.shift(1).loc[row.name]), axis=1)

In [ ]:
df_trajectories

In [ ]:
# save the final data frame with mapped model and weights
df_trajectories.to_csv('../../results/trajectories-mapped-powertrain-weight-grade-oct2021-sep2022.csv')